In [1]:
import os
import magenta.music as mm
from magenta.models.shared import sequence_generator_bundle

mm.notebook_utils.download_bundle("drum_kit_rnn.mag", "bundles")
bundle = sequence_generator_bundle.read_bundle_file(os.path.join("bundles", "drum_kit_rnn.mag"))

In [2]:
from magenta.models.drums_rnn import drums_rnn_sequence_generator

generator_map = drums_rnn_sequence_generator.get_generator_map()
generator = generator_map["drum_kit"](checkpoint = None, bundle = bundle)
generator.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmphdr62mk1\model.ckpt


In [3]:
from magenta.music import constants

# 1 bar primer - generate 3 bars to get 4 bars
num_bars = 3
qpm = 150

# 4 steps per quarter
secs_per_step = 60.0 / (qpm * generator.steps_per_quarter)
num_steps_per_bar = constants.DEFAULT_STEPS_PER_BAR
secs_per_bar = num_steps_per_bar * secs_per_step

print("Seconds per steps : {}".format(secs_per_step))
print("Seconds per bar : {}".format(secs_per_bar))

Seconds per steps : 0.1
Seconds per bar : 1.6


In [4]:
# Using a 1 bar primer
# PC# 26 : Acoustic Guitar (Steel)
# PC# 34 : Electric Bass (finger)
# PC# 42 : Viola
# PC# 61 : French Horn

primer_drums = mm.DrumTrack([frozenset(pitches) for pitches in 
                            [(42, 61), (), (26, ), (),
                             (42, 34, 61), (), (26, ), (),
                             (), (), (42, ), (), 
                             (42, 34), (), (26, 61), (), ]])
primer_sequence = primer_drums.to_sequence(qpm = qpm)
primer_start_time = 0
primer_end_time = primer_start_time + secs_per_bar

gen_start_time = primer_end_time
gen_end_time = gen_start_time + (secs_per_bar * num_bars)

print('Primer start : {} \t Primer end : {}'.format(primer_start_time, primer_end_time))
print('Generation start : {} \t Generation end : {}'.format(gen_start_time, gen_end_time))

Primer start : 0 	 Primer end : 1.6
Generation start : 1.6 	 Generation end : 6.4


In [9]:
from note_seq.protobuf import generator_pb2

temp = 0.9
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temp
print('Temperature : {}'.format(temp))

# Define generator
generator_options.generate_sections.add(start_time = gen_start_time, end_time = gen_end_time)

Temperature : 0.9


start_time: 1.6
end_time: 6.4

In [10]:
# Generating notesequence
sequence = generator.generate(primer_sequence, generator_options)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -46.300232 


In [11]:
from visual_midi import Plotter

midi = os.path.join('output', 'out_seq0.9.mid')
mm.midi_io.note_sequence_to_midi_file(sequence, midi)
print('Generated midi : {}'.format(os.path.abspath(midi)))

plot = os.path.join('output', 'out_plot.html')
pretty_midi = mm.midi_io.note_sequence_to_pretty_midi(sequence)
plotter = Plotter()
plotter.show(pretty_midi, plot)
print('Generated plot : {}'.format(os.path.abspath(plot)))

Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\out_seq0.9.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\out_plot.html
